# **TASK 1 — News Classification with BERT**

**1. Problem Statement & Objective**

Problem Statement:
News organizations publish thousands of articles daily. Manual categorization is inefficient and error-prone.

**Objective:**

To fine-tune a pre-trained Transformer model to automatically classify news articles into categories with high accuracy.

**2. Dataset Loading & Preprocessing**

We used a labeled news dataset containing article text and category labels.

**Steps:**

Loaded dataset using HuggingFace Datasets

Removed missing values

Tokenized text using BERT tokenizer

Applied padding and truncation

Converted labels to numeric format

Purpose: Convert raw text into numerical tensors understandable by the Transformer.


**3. Model Development & Training**

Model used: bert-base-uncased

**Architecture:**

Pre-trained BERT encoder

Classification head (Dense + Softmax)

Training process:

Loss: Cross-Entropy

Optimizer: AdamW

Epochs: 2

Batch training using Trainer API

Transfer learning allows faster convergence with limited data.

**4. Evaluation with Metrics**

**Metrics used:**

Accuracy

F1-Score

**Final Results:**

Metric	Value

Accuracy	92.05%

F1-Score	92.14%

**6. Visualization**

Published train model on gradio for dashboard.

**7. Final Summary / Insights**

The fine-tuned Transformer achieved high accuracy, proving that pre-trained language models can be efficiently adapted for domain-specific classification tasks.

In [ ]:
!pip install -q -U transformers datasets evaluate scikit-learn accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0 --index-url https://download.pytorch.org/whl/cpu

# ensure latest Transformers & Datasets
!pip install -U transformers datasets gradio scikit-learn


Found existing installation: torch 2.9.0+cpu
Uninstalling torch-2.9.0+cpu:
  Successfully uninstalled torch-2.9.0+cpu
Found existing installation: torchvision 0.24.0+cpu
Uninstalling torchvision-0.24.0+cpu:
  Successfully uninstalled torchvision-0.24.0+cpu
Found existing installation: torchaudio 2.9.0+cpu
Uninstalling torchaudio-2.9.0+cpu:
  Successfully uninstalled torchaudio-2.9.0+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.9.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.24.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.9.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.9 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.9.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl (184.4 MB)
Using cached https://download.pytorch.org/whl/c

In [ ]:
!pip install transformers datasets torch scikit-learn gradio

In [ ]:
# Task#1 BERT, News Dataset

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
from sklearn.metrics import f1_score
import evaluate

print("Device:", "cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
dataset = load_dataset("ag_news")

# OPTIONAL: SUBSET FOR SPEED
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(20000))   # 20k instead of 120k
dataset["test"]  = dataset["test"].shuffle(seed=42).select(range(2000))     # 2k instead of 7.6k

# Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding=False,
        max_length=128   # shorter for CPU speed
    )

tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])

data_collator = DataCollatorWithPadding(tokenizer)

# Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4
)

# Metrics
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1_score(labels, preds, average="macro")
    }

# Training Args
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=3e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=False,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train
trainer.train()

# Evaluate
results = trainer.evaluate()
print("\nFINAL METRICS:")
print(results)

# Save Model
trainer.save_model("./distilbert_agnews_model")
tokenizer.save_pretrained("./distilbert_agnews_model")

print("\nModel saved to ./distilbert_agnews_model")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 22.5 MB/s eta 0:00:00


Device: cpu


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.233700,0.266285,0.915000,0.915836


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.233700,0.266285,0.915000,0.915836
2,0.179600,0.272406,0.920500,0.921442


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



FINAL METRICS:
{'eval_loss': 0.27240559458732605, 'eval_accuracy': 0.9205, 'eval_f1': 0.9214415118922797, 'eval_runtime': 226.6821, 'eval_samples_per_second': 8.823, 'eval_steps_per_second': 0.551, 'epoch': 2.0}

Model saved to ./distilbert_agnews_model


In [ ]:
!pip install streamlit transformers torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.7 MB/s eta 0:00:00


In [ ]:
!pip install gradio transformers torch


In [ ]:
# For Streaming on Gradio

import gradio as gr
from transformers import pipeline

clf = pipeline("text-classification", model="./distilbert_agnews_model")

label_map = {
    "LABEL_0": "World",
    "LABEL_1": "Sports",
    "LABEL_2": "Business",
    "LABEL_3": "Sci/Tech"
}

def predict_topic(text):
    if not text.strip():
        return "Please enter a headline"

    result = clf(text)[0]
    topic = label_map[result["label"]]
    score = result["score"]

    return f"{topic}  (confidence: {score:.3f})"

app = gr.Interface(
    fn=predict_topic,
    inputs=gr.Textbox(label="Enter News Headline"),
    outputs=gr.Textbox(label="Predicted Topic"),
    title="News Topic Classifier - DistilBERT"
)

app.launch(share=True)


Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://76f217f3c9c557c42e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
